In [1]:
import pandas as pd
semester = list()
file_path = 'MS_Thesis_Repository_FSC.xlsx'
all_data = pd.read_excel(file_path, sheet_name=None)
thesis_data = all_data
for sheet_name, data in all_data.items():
    print("Sheet name:", sheet_name)
    semester.append(sheet_name)

Sheet name: Spring-2023
Sheet name: Fall-2022
Sheet name: Spring-2022
Sheet name: Fall-2021
Sheet name: Spring-2021
Sheet name: Fall-2020
Sheet name: Spring-2020
Sheet name: Fall-2019
Sheet name: Spring-2019


c:\Users\mustf\anaconda3\envs\py310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [2]:
import pandas as pd

file_path = 'MS_Thesis_Repository_FSC.xlsx'
all_data = pd.read_excel(file_path, sheet_name=None)

for sheet_name, data in all_data.items():
    processed_data = data.iloc[1:, [1, 3]]  
    processed_data.columns = ['Thesis Title', 'Thesis Abstract']
    
    output_path = f'{sheet_name}.xlsx'
    processed_data.to_excel(output_path, index=False)


c:\Users\mustf\anaconda3\envs\py310\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
import pandas as pd
import glob
import os


# List to hold data from each Excel file
dataframes = []

for file,data in all_data.items():
    # Read the Excel file into a DataFrame
    file = f'{file}.xlsx'
    df = pd.read_excel(file)
    # Append the DataFrame to the list
    dataframes.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Path for the resulting text file
txt_file_path = 'combined_output.txt'

# Write the combined DataFrame to a text file, separated by tabs
combined_df.to_csv(txt_file_path, sep='\t', index=False)

print(f"All files have been combined and saved to {txt_file_path}")


All files have been combined and saved to combined_output.txt


In [4]:

from langchain.embeddings.fastembed import FastEmbedEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
import tiktoken
from langchain.vectorstores import Qdrant
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOllama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA


path_f = 'combined_output.txt'

loader = TextLoader(path_f)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=7000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [5]:
print(type(texts))
print(texts[0])
print("[+]=====[+]")
print("[+]Done --> Splitting txt file to chunks")

<class 'list'>
page_content='Thesis Title\tThesis Abstract\n"21I-2191 Laraib Afzal\n\nProsodic alignment for Automatic dubbing\n\n"\tAutomatic dubbing is the process of replacing the audio track of a video with a different language. In automatic dubbing, prosodic alignment is used to match the suprasegmental features like timbre, prosody, duration, pauses and intonation of the original speech with synthesed speech, in order to produce a natural-sounding dubbed video. This is done by analyzing and mapping these features of the original and translated speech. Existing research on automatic dubbing lack to addresses these features in source video which impact the overall naturalness and fluency of Synthesized speech. To solve this we proposed end-to-end architecture, following modular approach, to generate high quality dubbed video. In this research, we mainly focus on TTS module by considering mentioned features for prosodic alignment in generated speech with synchronization of original 

In [6]:
with open(path_f, 'r') as file:
    content = file.read()

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens
    
print("[+]=====[+]")
numtoken = num_tokens_from_string(content, "cl100k_base")
print("Number Of Token: " + str(numtoken))

embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5", max_length=2048)

[+]=====[+]
Number Of Token: 139703


2024-06-15 07:47:40.192 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
qdrant = Qdrant.from_documents(
    texts,
    embeddings,
    path="./local_qdrant1",
    collection_name="db2",
)
print("[+]=====[+]")
print("[+]Done  --> Embedding the Chunks, and saving to a vectorstore")

question = input("Enter Any Question Related to the documents: ")
docs = qdrant.similarity_search(question)

print(docs[0].page_content)
len(docs)

[+]=====[+]
[+]Done  --> Embedding the Chunks, and saving to a vectorstore
Ramsha Amin (MS-CS)
Supervisor: Dr. Amna Basharat "	Semantic web is an extension of the current web in which semantic-based techniques are used to define and link data which provides better information integration and search experiences for the web users. In the domain of Islamic knowledge, especially focusing on the Quranic Tajweed which is the science of Quranic recitation, not enough work has been done for gathering all the knowledge related to the recitation rules of the Holy Quran in one place. Ontologies, are one of the tools frequently used for organizing knowledge which describes the terms of the domain and explains the relationships among them. In our study, we have developed an ontology-based Tajweed knowledge model by using Proteg ́ e framework and state-of-the-art semantic web technologies (OWL and SPARQL). METHONTOLOGY, an iterative design methodology was used for the ontology development that descr

4

In [8]:
embeddings

FastEmbedEmbeddings(model_name='BAAI/bge-base-en-v1.5', max_length=2048, cache_dir=None, threads=None, doc_embed_type='default', _model=<fastembed.text.text_embedding.TextEmbedding object at 0x000001E276BCA0E0>)

In [9]:
# Preparing our chat model to make the similarity search result more human-like
template = """[INST] <<SYS>> Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
<</SYS>>
{context}
Question: {question}
Helpful Answer:[/INST]"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)
print("[+]=====[+]")
print("[+]Done --> Preparing our chat model to make the similarity search result more human-like")

# Choosing the free llama 7B for the task (Excellent)
chat_model = ChatOllama(
    model="llama3",
    verbose=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)
print("[+]=====[+]")
print("[+]Done --> Choosing the free llama3 7B for the task (Excellent)")

# Adding a retrieval QA
qa_chain = RetrievalQA.from_chain_type(
    chat_model,
    retriever=qdrant.as_retriever(),
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)
print("[+]=====[+]")
print("[+]Done --> Adding a retrieval QA")
print("[+]Go ahead and use the bot to answer any question from your documents (use ctrl + c to end the bot chat)")

# We are now ready, and good to go to ask question based on our document, and get human like answer
while True:
    
    try:
        question = input("Enter Prompt: ")
        docs = qdrant.similarity_search(question)
        print("Question: " + question)
        print("Answer ->>> ")
        result = qa_chain({"query": question})
        print("\n")
    except KeyboardInterrupt:
        print("[+]==[+]")
        print("Bye! : Ending the Bot Chat")
        break


[+]=====[+]
[+]Done --> Preparing our chat model to make the similarity search result more human-like
[+]=====[+]
[+]Done --> Choosing the free llama3 7B for the task (Excellent)
[+]=====[+]
[+]Done --> Adding a retrieval QA
[+]Go ahead and use the bot to answer any question from your documents (use ctrl + c to end the bot chat)
Question: hello
Answer ->>> 
Hello!

It seems like you're looking for information related to the infrastructure less, flexible, self-configurable and is relatively small and less expense network. You also want to know more about efficient flexible periodic pattern mining, text classification using ensemble method, and predicting protein folding rates using machine learning.

In regards to your question, I can provide some general insights on these topics:

* Flying Ad-Hoc Networks (FANETs) are a type of infrastructure-less network that uses multiple Unmanned Air Vehicles (UAVs) to establish connectivity. FANETs have the potential to be used for various applicat